In [16]:
import sqlite3
import pandas as pd
import logging

# Setup logging
logging.basicConfig(
    filename="vendor_sales_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

In [17]:
conn = sqlite3.connect("inventory.db")

In [18]:
def ingest_db(df, table_name, engine):
    """Ingests a DataFrame into the SQLite database."""
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

In [19]:
conn.execute("""
CREATE TABLE IF NOT EXISTS purchases (
    VendorNumber TEXT,
    VendorName TEXT,
    Brand TEXT,
    Description TEXT,
    PurchasePrice REAL,
    Quantity INTEGER,
    Dollars REAL
);
""")

conn.execute("""
CREATE TABLE IF NOT EXISTS purchase_prices (
    Brand TEXT,
    Price REAL,
    Volume TEXT
);
""")

conn.execute("""
CREATE TABLE IF NOT EXISTS vendor_invoice (
    VendorNumber TEXT,
    Freight REAL
);
""")

conn.execute("""
CREATE TABLE IF NOT EXISTS sales (
    VendorNo TEXT,
    Brand TEXT,
    SalesQuantity INTEGER,
    SalesDollars REAL,
    SalesPrice REAL,
    ExciseTax REAL
);
""")


In [20]:
def create_vendor_summary(conn):
    """
    Merges multiple tables to generate a vendor performance summary.
    """
    query = """
    WITH FreightSummary AS (
        SELECT VendorNumber, SUM(Freight) AS FreightCost
        FROM vendor_invoice
        GROUP BY VendorNumber
    ),
    
    PurchaseSummary AS (
        SELECT 
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price AS ActualPrice,
            pp.Volume,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars
        FROM purchases p
        JOIN purchase_prices pp ON p.Brand = pp.Brand
        WHERE p.PurchasePrice > 0
        GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
    ),
    
    SalesSummary AS (
        SELECT 
            VendorNo,
            Brand,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesPrice) AS TotalSalesPrice,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    )
    
    SELECT 
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.Description,
        ps.PurchasePrice,
        ps.ActualPrice,
        ps.Volume,
        ps.TotalPurchaseQuantity,
        ps.TotalPurchaseDollars,
        ss.TotalSalesQuantity,
        ss.TotalSalesDollars,
        ss.TotalSalesPrice,
        ss.TotalExciseTax,
        fs.FreightCost
    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss 
        ON ps.VendorNumber = ss.VendorNo 
        AND ps.Brand = ss.Brand
    LEFT JOIN FreightSummary fs 
        ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.TotalPurchaseDollars DESC
    """
    
    return pd.read_sql_query(query, conn)


In [21]:
logging.info("Creating Vendor Summary Table...")
try:
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())
except Exception as e:
    logging.error(f"Error generating summary: {e}")
